# Velogames solver: Giro d'Italia 2023

A script to calculate the optimal team that could have been chosen for a given race in [Velogames fantasy cycling](https://www.velogames.com/)

This Julia script uses the [Gumbo](https://github.com/JuliaWeb/Gumbo.jl) and [Cascadia](https://github.com/Algocircle/Cascadia.jl) libraries to scrape rider data, and the [JuMP](https://jump.dev/JuMP.jl/stable/) optimisation library / [HiGHS](https://highs.dev/) solver to construct and solve a mixed-integer program (MIP) described below

In Velogames fantasy cycling, you must select a team of 9 riders, each with a specific cost based on their expected performance, spending no more than 100 points.

Each rider is classed as either an All-Rounder, a Climber, a Sprinter or is Unclassed. A team must contain 2 All-Rounders, 2 Climbers, 1 Sprinter and 3 Unclassed riders. The 9th selection can be from any of these categories.

At the end of the race, each rider will have accumulated a score based on their performance, and the aim is to pick a team with the highest combined score at the end of the race.

The optimisation problem can be stated as:

$maximise \sum_{j=1}^{n} x_j y_j$

$s.t.$

$\sum_{j=1}^{n} x_j=9$

$\sum_{j=1}^{n} x_j z_j \leq 100$

$\sum_{j=1}^{n} x_j a_j \geq 2$

$\sum_{j=1}^{n} x_j c_j \geq 2$

$\sum_{j=1}^{n} x_j s_j \geq 1$

$\sum_{j=1}^{n} x_j u_j \geq 3$

where $j=1...n$ is the set of all riders

$x_j\in[0,1]$ is a binary decision variable denoting if rider $j$ is chosen (1 for chosen, 0 for not chosen)

$z_j\in Z^+$ and $y_j\in Z^+$ are the cost and score parameters of rider $j$ respectively

$a_j\in[0,1]$, $c_j\in[0,1]$, $s_j\in[0,1]$ and $u_j\in[0,1]$ are binary parameters denoting if rider $j$ is an All-Rounder, Climber, Sprinter or Unclassed respectively, with the further parameter constraint that $a_i+c_i+s_i+u_i=1$ $\forall i=1...n$ (i.e. each rider is allocated to one and only one of the 4 categories) and by implication $\sum_{j=1}^{n} a_j+\sum_{j=1}^{n} c_j+\sum_{j=1}^{n} s_j+\sum_{j=1}^{n} u_j=n$ (i.e. the sum of the number of riders in each category is equal to the total number of riders)

## Load libraries


In [1]:
using Statistics

# include all files in src directory
source_files = readdir("src")
for file in source_files
    include("src/$file")
end

In [2]:
FORM_WEIGHT = 0.5

rider_df = getvgriders("https://www.velogames.com/italy/2023/riders.php")

mycols = [:mountain, :gc, :sprint, :overall]
# getpcsranking for each col, filter the dataframe to only the points and riderkey columns, and rename the points column to the col name.
pcs_dfs = map(mycols) do col
    rename(getpcsranking(col), :points => col)[:, [col, :riderkey]]
end

# use reduce to join all dataframes in pcs_dfs on the riderkey column
pcs_df = reduce((x, y) -> outerjoin(x, y, on=:riderkey), pcs_dfs)

# join the velogames and pcs dataframes on the riderkey column
rider_df = leftjoin(rider_df, pcs_df, on=:riderkey)
rider_df = coalesce.(rider_df, 0)

# create calcpcsscore column which is the sum of All Rounder * gc, Sprinter * sprint, Climber * mountain, and Unclassed * overall
rider_df.calcpcsscore = rider_df.allrounder .* rider_df.gc .+ rider_df.sprinter .* rider_df.sprint .+ rider_df.climber .* rider_df.mountain .+ rider_df.unclassed .* rider_df.overall

# create calc_score column which is a weighted average of the calcpcsscore and points columns, adjusted for the difference in magnitude
ADJ_FACTOR = mean(rider_df.points) / mean(rider_df.calcpcsscore)
rider_df.calc_score = (1 - FORM_WEIGHT) .* ADJ_FACTOR .* rider_df.calcpcsscore .+ FORM_WEIGHT .* rider_df.points

176-element Vector{Float64}:
 235.00121509273075
 288.6208484331699
 241.40667235131102
 399.97537838413984
 279.5511617991899
 248.90277126412278
 272.1491153272223
 185.64229375399702
  91.29458537625239
 121.44127051801321
  96.07260712001704
  97.27021956938819
 148.73001492219143
   ⋮
   0.0
   6.0
   6.0
   6.0
   6.0
   4.0
   4.0
  14.0
  72.0
  10.5
   3.0
   3.0

In [3]:
model_results = build_model_stage(rider_df)

Running HiGHS 1.5.1 [date: 1970-01-01, git hash: 93f1876e4]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
6 rows, 176 cols, 528 nonzeros
6 rows, 103 cols, 266 nonzeros
6 rows, 102 cols, 264 nonzeros

Solving MIP model with:
   6 rows
   102 cols (86 binary, 16 integer, 0 implied int., 0 continuous)
   264 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   8447            -inf                 inf        0      0      0         0     0.0s
 R       0       0         0   0.00%   2273.685172     2189.042336        3.87%        0      0      0        10     0.0s

61.8% inactive integer columns, restarting
Model after restart has 6 rows, 39 cols (39 bin., 0 int., 0 impl., 0 cont.), and 104 nonzeros

         0       0         0   0.00

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, ["Michael Matthews", "Mads Pedersen", "Kaden Groves", "Remco Evenepoel", "Primož Roglič", "Tao Geoghegan Hart", "João Almeida", "Geraint Thomas", "Hugh Carthy", "Thibaut Pinot"  …  "Niklas Märkl", "Florian Stork", "Martijn Tusveld", "Lukas Pöstlberger", "Callum Scotson", "Campbell Stewart", "Amanuel Ghebreigzabhier", "Daan Hoole", "Alex Kirsch", "Otto Vergaerde"]
And data, a 176-element Vector{Float64}:
  0.0
 -0.0
  1.0
  1.0
  0.0
 -0.0
  1.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [4]:
# total cost
rider_df.cost .* model_results.data |> sum

100.0

In [5]:
# selected riders
rider_df[!, :chosen] = model_results.data .|> !iszero
chosen_team = filter(:chosen => ==(true), rider_df)
chosen_team[:, [:rider, :team, :class_raw, :selected, :points, :calcpcsscore, :cost]]

Row,rider,team,class_raw,selected,points,calcpcsscore,cost
,String,String,String,String,Float64,Float64,Int64
1,Kaden Groves,Alpecin-Deceuninck,Sprinter,21.4%,342.0,391.0,8
2,Remco Evenepoel,Soudal - Quick Step,All Rounder,68.7%,402.0,1105.0,26
3,João Almeida,UAE Team Emirates,All Rounder,38.6%,285.0,720.0,16
4,Jay Vine,UAE Team Emirates,Climber,21.8%,185.0,466.0,12
5,Stefan Küng,Groupama - FDJ,Unclassed,19.2%,174.0,1100.7,8
6,Andreas Leknessund,Team DSM,Unclassed,5.6%,255.0,519.0,6
7,Filippo Ganna,INEOS Grenadiers,Unclassed,39.3%,264.0,1036.3,10
8,Vincenzo Albanese,EOLO-Kometa,Sprinter,5.8%,336.0,164.0,6
9,Koen Bouwman,Jumbo-Visma,Climber,3.0%,146.0,402.0,8
